In [1]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [2]:
file  = open('data.txt', 'r', encoding = "utf8")

In [4]:
lines=[]
for i in file:
    lines.append(i)

In [6]:
data=""
for i in lines:
    data = ' '.join(lines)

In [8]:
 data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')

In [9]:
data = data.split()
data = ' '.join(data)

In [12]:
data[:1000]

"THE ADVENTURES OF SHERLOCK HOLMES Arthur Conan Doyle Table of contents A Scandal in Bohemia The Red-Headed League A Case of Identity The Boscombe Valley Mystery The Five Orange Pips The Man with the Twisted Lip The Adventure of the Blue Carbuncle The Adventure of the Speckled Band The Adventure of the Engineer's Thumb The Adventure of the Noble Bachelor The Adventure of the Beryl Coronet The Adventure of the Copper Beeches A SCANDAL IN BOHEMIA Table of contents Chapter 1 Chapter 2 Chapter 3 CHAPTER I To Sherlock Holmes she is always the woman. I have seldom heard him mention her under any other name. In his eyes she eclipses and predominates the whole of her sex. It was not that he felt any emotion akin to love for Irene Adler. All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind. He was, I take it, the most perfect reasoning and observing machine that the world has seen, but as a lover he would have placed himself in a false positio

In [13]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

In [14]:
pickle.dump(tokenizer, open('token.pkl', 'wb'))

In [15]:
sequence_data = tokenizer.texts_to_sequences([data])[0]

In [17]:
sequence_data[:10]

[1, 1561, 5, 129, 34, 647, 4498, 4499, 226, 5]

In [18]:
len(sequence_data)

105879

In [19]:
vocab_size = len(tokenizer.word_index)+1

In [20]:
vocab_size

8200

In [33]:
sequence=[]
for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequence.append(words)

In [34]:
len(sequence)

105876

In [35]:
sequence = np.array(sequence)

In [36]:
sequence

array([[   1, 1561,    5,  129],
       [1561,    5,  129,   34],
       [   5,  129,   34,  647],
       ...,
       [  28,    1, 8198, 8199],
       [   1, 8198, 8199, 3187],
       [8198, 8199, 3187, 3186]])

In [37]:
X=[]
y=[]

for i in sequence:
    X.append(i[0:3])
    y.append(i[3])

In [40]:
X=np.array(X)
y=np.array(y)

In [41]:
y = to_categorical(y, num_classes=vocab_size)

In [43]:
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

Building the LSTM Model

In [45]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length = 3))
model.add(LSTM(1000, return_sequences = True))
model.add(LSTM(1000))
model.add(Dense(1000, activation='relu'))
model.add(Dense(vocab_size, activation="softmax"))

In [46]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3, 10)             82000     
_________________________________________________________________
lstm (LSTM)                  (None, 3, 1000)           4044000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 1000)              8004000   
_________________________________________________________________
dense (Dense)                (None, 1000)              1001000   
_________________________________________________________________
dense_1 (Dense)              (None, 8200)              8208200   
Total params: 21,339,200
Trainable params: 21,339,200
Non-trainable params: 0
_________________________________________________________________


In [47]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("next_words.h5", monitor="loss", verbose = 1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer = Adam(learning_rate = 0.001))
model.fit(X, y, epochs=2, batch_size=64, callbacks=[checkpoint])

Epoch 1/2
1655/1655 [==============================] - ETA: 0s - loss: 6.3547
Epoch 00001: loss improved from inf to 6.35469, saving model to next_words.h5
1655/1655 [==============================] - 559s 337ms/step - loss: 6.3547
Epoch 2/2
1655/1655 [==============================] - ETA: 0s - loss: 5.7760
Epoch 00002: loss improved from 6.35469 to 5.77597, saving model to next_words.h5
1655/1655 [==============================] - 554s 335ms/step - loss: 5.7760
